In [ ]:
from bs4 import BeautifulSoup                                       # HTML və XML-dən məlumat çıxarmaq üçün
from selenium import webdriver                                      # Brauzeri avtomatik idarə etmək üçün
from selenium.webdriver.chrome.service import Service               # ChromeDriver-i başlatmaq üçün
from selenium.webdriver.common.by import By                         # HTML elementlərini axtarmaq üçün (məs: By.CLASS_NAME)
from webdriver_manager.chrome import ChromeDriverManager            # ChromeDriver versiyasını avtomatik endirmək üçün
from selenium.webdriver.chrome.options import Options               # Chrome brauzerinə xüsusi ayarlar vermək üçün
from selenium.webdriver.support.ui import WebDriverWait             # Elementlərin yüklənməsini gözləmək üçün
from selenium.webdriver.support import expected_conditions as EC    # Müəyyən şərtlər ödənənə qədər gözləmək üçün
from selenium.common.exceptions import TimeoutException             # Vaxt aşımı xətasını idarə etmək üçün
from selenium.webdriver.common.keys import Keys                     # Klaviatura düymələrini simulyasiya etmək üçün (məs: ENTER)
import time                                                         # Kodda gecikmə vermək üçün (məs: time.sleep)
import pandas as pd                                                 # Verilənlərlə işləmək üçün (DataFrame, CSV və s.)
import re                                                           # Mətn təmizləmək, regex (müntəzəm ifadələr) ilə işləmək üçün
import pyodbc                                                       # SQL Server (və digər ODBC dəstəkləyən verilənlər bazaları) ilə bağlantı qurmaq üçün
from sqlalchemy import create_engine                                # SQLAlchemy ilə verilənlər bazasına bağlantı yaratmaq və pandas ilə işləmək üçün
import requests
import os
import base64
import numpy as np
import json
import pandas as pd
import queue
from concurrent.futures import ThreadPoolExecutor
from threading import Thread
from bs4 import NavigableString
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


raw_data = queue.Queue()
cleaned_data = queue.Queue()
list_data = []



def scrap():
    base_urls = ['https://arenda.az/filtirli-axtaris/?home_search=1&lang=1&site=1&home_s=1&elan_novu%5B1%5D=1&elan_novu%5B2%5D=2&elan_novu%5B3%5D=3&emlak_novu%5B370%5D=370&emlak_novu%5B371%5D=371&emlak_novu%5B5%5D=5&emlak_novu%5B861%5D=861&emlak_novu%5B94%5D=94&emlak_novu%5B8%5D=8&emlak_novu%5B270%5D=270&emlak_novu%5B854%5D=854&price_min=&price_max=&axtar=&sahe_min=&sahe_max=&mertebe_min=&mertebe_max=&y_mertebe_min=&y_mertebe_max=']

    def scrape_url(current_url):
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.maximize_window()
        # linklerin tekrarlanmasinin qarsisini almaq
        see_link = set()
        print(f"\n{current_url} üzrə toplanir")
        
        for i in range(1, 1001):
            # Səhifə URL-ni qurmaq
            url = f"https://arenda.az/filtirli-axtaris/{i}/?home_search=1&lang=1&site=1&home_s=1&elan_novu%5B1%5D=1&elan_novu%5B2%5D=2&elan_novu%5B3%5D=3&emlak_novu%5B370%5D=370&emlak_novu%5B371%5D=371&emlak_novu%5B5%5D=5&emlak_novu%5B861%5D=861&emlak_novu%5B94%5D=94&emlak_novu%5B8%5D=8&emlak_novu%5B270%5D=270&emlak_novu%5B854%5D=854&price_min=&price_max=&axtar=&sahe_min=&sahe_max=&mertebe_min=&mertebe_max=&y_mertebe_min=&y_mertebe_max="
            # Səhifə yükləmə cəhdləri
            for attempt in range(2):  
                try:
                    driver.set_page_load_timeout(45)
                    driver.get(url)
                    break
                except TimeoutException:
                    print(f"{i}. səhifə - Cəhd {attempt+1}: vaxt limiti keçdi")
                    time.sleep(5)

            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "[id^='elan_'] > a")))
            except TimeoutException:
                print(f"{i}. səhifə yüklənmədi, keçilir...")
                continue

            time.sleep(1)

            # Elan linklərinin tapılması
            elements = driver.find_elements(By.CSS_SELECTOR, "[id^='elan_'] > a")
            links = [e.get_attribute("href") for e in elements if e.get_attribute("href")]
            
            if not links:
                print(f"{current_url} - Səhifə {i} üçün elan tapılmadı")
                continue

            # Tapılan elan linklerinden melumatlarin toplanmasi
            for link in links:
                if not ("/satilir" in link or "/kiraye" in link):
                    continue

                # link gorulubse , kec
                if link in see_link:
                    continue

                # Gorulmus linkler see_link - e elave edilir
                see_link.add(link)

                # Elan səhifəsinə daxil olmaq
                for attempt in range(2):
                    try:
                        driver.set_page_load_timeout(45)
                        driver.get(link)
                        break
                    except TimeoutException:
                        print(f"Elan səhifəsi {link} - Cəhd {attempt+1}: vaxt limiti keçdi")
                        time.sleep(5)
                
                # Elanın məlumatlarının yüklənməsini gözləyir
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
                except TimeoutException:
                    print(f"Elan yüklənmədi: {link}")
                    continue        
            

                #--------------------------------------------------------------------------
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                data = {"URL": link}


                #--------------------------------------------------------------------------
                # ID
                ann_id = soup.find('div', class_='elan_e_kodu').find('strong')
                data['ListingID'] = ann_id.text.strip() if ann_id else None


                #--------------------------------------------------------------------------
                # Elan adi 
                announce = soup.find("h1", {"class":"elan_title"})
                data['ListingName'] = announce.text.strip() if announce else None


                #--------------------------------------------------------------------------
                # Labels
                labels = soup.find("ul", class_="elan_adr_list full")  # tək 'ul' tapırıq, ona görə find istifadə olunur
                data["Labels"] = [li.get_text(strip=True) for li in labels.find_all("li")] if labels else []


                #--------------------------------------------------------------------------
                # Category 
                category = soup.find("h2", {"class":"full elan_in_title_link elan_main_title"})
                data["CategoryID"] = category.text.strip() if category else None


                #-------------------------------------------------------------------------
                # Tarix
                date = soup.find("p", string=lambda text: text and "Elanın tarixi" in text)
                data['ListingDate'] = date.text.strip() if date else None


                #--------------------------------------------------------------------------
                # Aciqlama
                description = soup.find("div", {"class":"full elan_info_txt"})
                data['Description'] = description.text.strip() if description else None


                #--------------------------------------------------------------------------
                # unvan 
                address = soup.find("span", {"class":"elan_unvan_txt"})
                data['Address'] = address.text.strip() if address else None


                #--------------------------------------------------------------------------
                # Melumat bloku
                ul = soup.find('ul', {"class":"full elan_property_list"})
                links = ul.find_all('a')

                data['CountRoom'] = None
                data['FieldAreaSqm'] = None
                data['AreaInSot'] = None
                data['Floor'] = None
                data['Title'] = "yoxdur"  # default olaraq kupça yoxdur

                for a in links:
                    text = a.text.strip()

                    if "otaq" in text and data['CountRoom'] is None:
                        data['CountRoom'] = text

                    if "sot" in text and data['AreaInSot'] is None:
                        data['AreaInSot'] = text

                    if ("mərtəbəli" in text or "mərtəbə" in text) and data['Floor'] is None:
                        data['Floor'] = text

                    if "m2" in text and data['FieldAreaSqm'] is None:
                        data['FieldAreaSqm'] = text

                    if "Kupça (Çıxarış)" in text:
                        data['Title'] = "var"


                #--------------------------------------------------------------------------
                # Qiymet
                price = soup.find("p", {"class":"full"})
                data['Price'] = price.text.strip() if price else None

                # price_p = soup.find("p", class_="full m-0")

                # if price_p:
                #     for content in price_p.contents:
                #         if isinstance(content, NavigableString):
                #             data['Price'] = content.strip()
                #             break
                # else:
                #     data['Price'] = None


                #--------------------------------------------------------------------------
                # 1 kv.m
                pricesqm = soup.find("span", {"class":" elan_new_prop_span elan_new_prop_top_span"})
                data['PricePerSqm'] = pricesqm.strip() if pricesqm else None


                #--------------------------------------------------------------------------
                # nomre
                salesperson = soup.find("div", {"class":"new_elan_user_info full"}).find('p')
                data['SalesPerson'] = salesperson.text.strip() if salesperson else None

                #---------------------------------------------------------------------------
                # contact 
                contact = soup.find("a", {"class":"elan_in_tel"})
                data['Contact'] = contact.text.strip() if contact else None


                #----------------------------------------------------------------------------
                lat_input = soup.find("input", {"id": "lat"})
                lon_input = soup.find("input", {"id": "lon"})

                data['Latitude'] = lat_input.get('value') if lat_input else None
                data['Longitude'] = lon_input.get('value') if lon_input else None


                #----------------------------------------------------------------------------
                # images
                images = soup.find_all("img", {"class": "translate"})
                data['Images'] = [img['src'] for img in images if img.get('src')]


                #--------------------------------------------------------------------------
                list_data.append(data)

                df = pd.DataFrame([data])
                raw_data.put(df)

                
                #--------------------------------------------------------------------------
                print(f"Sehife {i}. Ümumi {len(list_data)} elan toplandı.")


        driver.quit()
    with ThreadPoolExecutor(max_workers=len(base_urls)) as executor:
        executor.map(scrape_url, base_urls)



def preprocess(raw):
    df = raw.copy()

    df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce')
    df['ListingID'] = df['ListingID'].apply(lambda x: '505' + str(int(x)) if pd.notna(x) else x) 
    df['ListingID'] = pd.to_numeric(df['ListingID'], errors='coerce').astype("Int64")


    df['Date'] = pd.to_datetime(
    df['ListingDate'].str.replace('Elanın tarixi: ', '', regex=False), format='%d.%m.%Y')

    # 1. SalesType sutunu yaratmaq
    df['SalesType'] = df['ListingName'].apply(lambda x: 0 if 'Kirayə' in str(x) else 1 if 'Satılır' in str(x) else np.nan)
    df['SalesType'] = df['SalesType'].astype('Int8')  # bit üçün Int8 istifadə edirik

    # 2. DailyRent sütunu: case-insensitive və sadə məntiq
    df['DailyRent'] = df['ListingName'].apply(
        lambda x: 1 if 'günlük' in str(x).lower() else 0 if 'aylıq' in str(x).lower() else np.nan)
    df['DailyRent'] = df['DailyRent'].astype('Int64')

    # 3. CategoryID-ni dəyişdirmək
    category_mapping = {
        'Yeni Tikili': 2,
        'Köhnə Tikili': 3,
        'Həyət evi/villa': 4,
        'Bağ evi': 4,
        'Obyekt': 8,
        'Ofis': 5,
        'Torpaq': 7,
        'Qaraj': 6}

    # Mətnləri dəyiş
    for key, value in category_mapping.items():
        df.loc[df['CategoryID'].astype(str).str.contains(key, na=False), 'CategoryID'] = value
    df['CategoryID'] = pd.to_numeric(df['CategoryID'], errors='coerce').astype('Int64')

    # 4. Description sutunundan emoji və əlavə simvolları silmək
    df['Description'] = df['Description'].apply(lambda x: 
        re.sub(r'[^\w\s\-.,!?()]+', '', str(x), flags=re.UNICODE).strip() if pd.notna(x) else x)

    # 5. CountRoom - "otaq" sözünü silmək
    df['CountRoom'] = df['CountRoom'].apply(lambda x: 
        int(re.sub(r'otaq', '', str(x), flags=re.IGNORECASE).strip()) if pd.notna(x) and 
        re.sub(r'otaq', '', str(x), flags=re.IGNORECASE).strip().isdigit() else np.nan)
    df['CountRoom'] = df['CountRoom'].astype('Int64')

    # 6. FieldAreaSqm - "m2" silmək və float64-ə çevirmək
    # df['FieldAreaSqm'] = df['FieldAreaSqm'].apply(lambda x: 
    #     float(re.sub(r'm2?', '', str(x), flags=re.IGNORECASE).strip()) if pd.notna(x) and 
    #     re.sub(r'm2?', '', str(x), flags=re.IGNORECASE).strip().replace('.','').isdigit() else np.nan)
    # df['FieldAreaSqm'] = df['FieldAreaSqm'].astype('float64')

    df['FieldAreaSqm_clean'] = (
        df['FieldAreaSqm']
        .astype(str)
        .str.replace(r'[^0-9.]', '', regex=True)
        .replace(r'^$', np.nan, regex=True))
    df['FieldAreaSqm_clean'] = pd.to_numeric(df['FieldAreaSqm_clean'], errors='coerce')
    df['FieldAreaSqm'] = df['FieldAreaSqm_clean'].astype('float64')


    # 7. Floor sutunundan FloorNumber və TotalFloors yaratmaq
    floor_info = df['Floor'].apply(lambda x: 
        re.search(r'(\d+)\s*/\s*(\d+)', str(x)).groups() if pd.notna(x) and 
        re.search(r'(\d+)\s*/\s*(\d+)', str(x)) else (np.nan, np.nan))

    df['FloorNumber'] = [int(info[0]) if info[0] is not np.nan else np.nan for info in floor_info]
    df['TotalFloors'] = [int(info[1]) if info[1] is not np.nan else np.nan for info in floor_info]

    df['FloorNumber'] = df['FloorNumber'].astype('Int64')
    df['TotalFloors'] = df['TotalFloors'].astype('Int64')

    # Floor sutununu silmək
    df = df.drop('Floor', axis=1)

    # 8. Title sutununu boolean-a çevirmək
    df['Title'] = df['Title'].apply(lambda x: False if str(x).lower() in ['yoxdur', 'nan', 'none'] else True)
    df['Title'] = df['Title'].astype('bool')

    df['Price'] = pd.to_numeric(
        df['Price'].astype(str).str.replace(r'[^\d.]', '', regex=True), 
        errors='coerce').astype('Int64')

    # 10. PricePerSqm sutununu təmizləmək
    df['PricePerSqm'] = df['PricePerSqm'].apply(lambda x: 
        int(''.join(re.findall(r'\d+', str(x))[:2])) if pd.notna(x) and 
        re.findall(r'\d+', str(x)) else np.nan)
    df['PricePerSqm'] = df['PricePerSqm'].astype('Int64')

    # 11. SalesPerson sutunundan SellerType yaratmaq
    df['SellerType'] = df['SalesPerson'].apply(lambda x: 
        1 if pd.notna(x) and '(Əmlak sahibi)' in str(x) else 
        2 if pd.notna(x) and '(Vasitəçi)' in str(x) else 3)
    df['SellerType'] = df['SellerType'].astype('Int64')


    df['AreaInSot'] = pd.to_numeric(df['AreaInSot'].str.lower().str.replace('sot', '').str.replace(',', '.').str.strip(),errors='coerce')

    #-----------------------------------------------------------------------
    city_map = {
        "Ağcabədi": 17, "Ağdam": 18, "Ağdaş": 19, "Ağstafa": 20, "Ağsu": 21,
        "Astara": 22, "Bakı": 23, "Balakən": 24, "Beyləqan": 25, "Bərdə": 26,
        "Biləsuvar": 27, "Cəlilabad": 28, "Daşkəsən": 29, "Gədəbəy": 30,
        "Gəncə": 31, "Göyçay": 32, "Göygöl": 33, "Hacıqabul": 34, "Xaçmaz": 35,
        "Xankəndi": 36, "Xırdalan": 37, "Xızı": 38, "Xudat": 39, "İmişli": 40,
        "İsmayıllı": 41, "Kürdəmir": 42, "Qax": 43, "Qazax": 44, "Qəbələ": 45,
        "Qobustan": 46, "Quba": 47, "Qusar": 48, "Lerik": 49, "Lənkəran": 50,
        "Masallı": 51, "Mingəçevir": 52, "Naxçıvan": 53, "Naxçıvan MR": 54,
        "Neftçala": 55, "Oğuz": 56, "Saatlı": 57, "Sabirabad": 58, "Salyan": 59,
        "Samux": 60, "Siyəzən": 61, "Sumqayıt": 62, "Şabran": 63, "Şamaxı": 64,
        "Şəki": 65, "Şəmkir": 66, "Şirvan": 67, "Tərtər": 68, "Tovuz": 69,
        "Ucar": 70, "Yardımlı": 71, "Yevlax": 72, "Zaqatala": 73, "Zərdab": 74
    }

    metro_mapping = {
        "20 Yanvar": 1, "28 May": 2, "8 Noyabr": 3, "Avtovağzal": 4, "Azadlıq": 5, "Bakmil": 6,
        "Dərnəgül": 7, "Elmlər Akademiyası": 8, "Əhmədli": 9, "Gənclik": 10, "Həzi Aslanov": 11,
        "İçərişəhər": 12, "İnşaatçılar": 13, "Koroğlu": 14, "Memar Əcəmi": 15, "Nəriman Nərimanov": 16,
        "Nəsimi": 17, "Neftçilər": 18, "Nizami": 19, "Qara Qarayev": 20, "Sahil": 21,
        "Xətai": 22, "Xalqlar dostluğu": 23, "Xocaəsən": 24, "Ulduz": 25, "Cəfər Cabbarlı": 26
    }

    settlement_mapping = {
        "Abşeron r.": 1, "Aşağı Güzdək": 2, "Atyalı": 3, "Ceyranbatan qəs.": 4, "Çiçək qəs.": 5, "Digah": 6,
        "Fatmayi": 7, "Göradil": 8, "Hökməli": 9, "Köhnə Corat": 10, "Qobu": 11, "Masazır": 12,
        "Mehdiabad": 13, "Məmmədli": 14, "Novxanı": 15, "Pereküşkül": 16, "Saray": 17, "Yeni Corat": 18,
        "Zuğulba": 19, "Binəqədi r.": 20, "Alatava 2": 21, "28 May qəs.": 22, "6-cı mikrorayon": 23,
        "7-ci mikrorayon": 24, "8-ci mikrorayon": 25, "9-cu mikrorayon": 26, "Biləcəri qəs.": 27,
        "Binəqədi qəs.": 28, "Xocəsən": 29, "Xutor qəsəbəsi": 30, "Rəsulzadə qəs.": 31, "Sulutəpə qəs.": 32,
        "Xətai r.": 33, "Ağ şəhər": 34, "Əhmədli": 35, "H.Aslanov qəs.": 36, "Köhnə Günəşli qəs.": 37,
        "NZS": 38, "Xəzər r.": 39, "Binə qəs.": 40, "Buzovna": 41, "Dübəndi": 42, "Gürgən": 43,
        "Qala": 44, "Mərdəkan": 45, "Şağan": 46, "Şimal qres": 47, "Şüvəlan": 48, "Türkan": 49,
        "Zirə": 50, "Qaradağ qəs.": 51, "Ələt": 52, "Baş ələt": 52, "Qızıldaş qəs.": 53, "Qobustan qəs.": 54, "Lökbatan qəs.": 55,
        "Müşfiqabad qəs.": 56, "Puta qəs.": 57, "Sahil qəs.": 58, "Səngəçal qəs.": 59, "Şubani qəs.": 60, "Nərimanov r.": 61,
        "Böyükşor qəs.": 62, "Nəsimi r.": 63, "1-ci mikrorayon": 64, "2-ci mikrorayon": 65, "3-cü mikrorayon": 66,
        "4-cü mikrorayon": 67, "5-ci mikrorayon": 68, "Kubinka": 69, "Nizami r.": 70, "8-ci kilometr": 71,
        "Keşlə qəs.": 72, "Pirallahı": 73, "Sabunçu r.": 74, "Albalılıq": 75, "Bakıxanov qəs.": 76,
        "Balaxanı qəs.": 77, "Bilgəh qəs.": 78, "Kürdəxanı qəs.": 79, "Maştağa qəs.": 80, "Nardaran qəs.": 81, "Pirşağı qəs.": 82,
        "Ramana qəs.": 83, "Sabunçu qəs.": 84, "Savalan qəs.": 85, "Yeni Balaxanı": 86, "Yeni Ramana": 87, "Zabrat qəs.": 88,
        "Səbail r.": 89, "20-ci sahə": 90, "Badamdar qəs.": 91, "Bayıl qəs.": 92, "Bibi Heybət qəs.": 93, "Şıxov": 94,
        "Suraxanı r.": 95, "Bahar qəs.": 96, "Bülbülə qəs.": 97, "Dədə Qorqud qəs.": 98, "Əmircan qəs.": 99, "Günəşli": 100,
        "Hövsan qəs.": 101, "Qaraçuxur": 102, "Massiv A": 103, "Massiv B": 104, "Massiv D": 105,
        "Massiv G": 106, "Massiv V": 107, "Suraxanı qəs.": 108, "Şərq": 109, "Yeni Günəşli qəs.": 110,
        "Yeni Suraxanı qəs.": 111, "Zığ qəs.": 112, "Yasamal r.": 113, "Yasamal qəs.": 114, "Yeni Yasamal qəs.": 115, "Kimya şəhərciyi": 117, "Məhəmmədli": 119, 
        "Papanin": 122, "Qara şəhər": 123, "Ümid": 125, "Zaqulba": 128, "Alatava": 130, "Albalılıq": 131, 
        "Güzdək": 132, "Ceyildağ qəs.": 133, "Heybət qəs.": 134, "Qarakosa qəs.": 135, "Qurd Qapısı": 136 , 
        "Çilov qəs.": 137, "Şamaxinka": 138, "Şıxlar qəs.": 139, "Şonqar qəs.": 140, "Xocəsən qəs.": 141, 
        "Korgöz qəs.": 142, "Kotal qəs.": 143, "UPD": 144, "Yeni Ələt qəs.": 145, "kənd Əhmədli": 146}

    def extract_city_name(listing_name):
        listing = str(listing_name).lower()
        cities_sorted = sorted(city_map.keys(), key=len, reverse=True)

        for city in cities_sorted:
            if city.lower() in listing:
                return city
        return None

    def extract_metro_name(listing_name):
        listing = str(listing_name).lower()

        for metro_name in metro_mapping:
            metro_patterns = [
                f"{metro_name.lower()} metrosu",
                f"{metro_name.lower()} metro",
                f"{metro_name.lower()},"
            ]
            for pattern in metro_patterns:
                if pattern in listing:
                    return metro_name
            if metro_name.lower() in listing:
                return metro_name
        return None

    def extract_settlement_name(listing_name):
        listing = str(listing_name).lower()
        settlements_sorted = sorted(settlement_mapping.keys(), key=len, reverse=True)

        # Əvvəlcə qəsəbə axtar
        for settlement_name in settlements_sorted:
            if "qəs" in settlement_name.lower() or "mikrorayon" in settlement_name.lower() or "massiv" in settlement_name.lower():
                if settlement_name.lower() in listing:
                    return settlement_name

        # Sonra rayon axtar
        for settlement_name in settlements_sorted:
            if " r." in settlement_name:
                rayon_name = settlement_name.replace(" r.", "").lower()
                if f"{rayon_name} rayonu" in listing or f"{rayon_name} r." in listing:
                    return settlement_name
        return None

    # ------------------- DataFrame tətbiqi -------------------

    # df sənin məlumat DataFrame-indir
    df['CityName'] = df['ListingName'].apply(extract_city_name)
    df['MetroName'] = df['ListingName'].apply(extract_metro_name)
    df['SettlementName'] = df['ListingName'].apply(extract_settlement_name)

    # ID-ləri map et
    df['CityID'] = df['CityName'].map(city_map).fillna(23).astype(int)  # default: Bakı
    df['MetroID'] = df['MetroName'].map(metro_mapping).astype('Int64')  # nullable
    df['SettlementsID'] = df['SettlementName'].map(settlement_mapping).astype('Int64')

    # Lazımsız kolonları sil
    df.drop(['CityName', 'MetroName', 'SettlementName'], axis=1, inplace=True)

    #------------------------------------------------------------------------

    df['Currency'] = 'AZN'


    # 12. Latitude və Longitude null olan sətirləri silmək
    df = df.dropna(subset=['Latitude', 'Longitude'])

    # Index-i sıfırlamaq
    df = df.reset_index(drop=True)

    return df


def cleaner():
    while True:
        raw = raw_data.get()        
        cleaned = preprocess(raw)        
        cleaned_data.put(cleaned)       
        raw_data.task_done()



def inserter():
    while True:
        cleaned = cleaned_data.get()
        insert_data(cleaned)
        cleaned_data.task_done()




def insert_data(df):    
    # Database bağlantısı
    try:
        conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=DESKTOP-KIDL0VQ\\SQLEXPRESS;" 
            "DATABASE=RealEstateDB;"
            "Trusted_Connection=yes;")
        cursor = conn.cursor()
        print("Database Connection")
    except Exception as e:
        print(f"Database Connect Xeta: {e}")
        return False

    print(f"Orijinal məlumat sayı: {len(df)}")
    df = df.drop_duplicates(subset=['ListingID'])
    print(f"Dublikatlar silindikdən sonra: {len(df)}")

    # null deyerleri ucun
    def safe_value(value):
        if pd.isna(value):
            return None
        return value

    # datalar database-de yoxlanir
    def is_listing_exists(listing_id, table_name):
        try:
            query = f"SELECT COUNT(*) FROM Dynamic.{table_name} WHERE ListingID = ?"
            cursor.execute(query, (listing_id,))
            result = cursor.fetchone()
            return result[0] > 0
        except Exception as e:
            print(f"Yoxlama xətası {table_name} - {listing_id}: {e}")
            return False

    stats = {
        'apartment': 0,
        'house': 0,
        'office': 0,
        'garage': 0,
        'land': 0,
        'commercial': 0,
        'estate_listing': 0,
        'seller_info': 0,
        'url': 0,
        'labels': 0,
        'description': 0,
        'listing_images': 0 }

    try:        
        estate_query = """
        INSERT INTO Dynamic.EstateListing (ListingID, ListingName, SalesType, DailyRent, CategoryID, SellerType, Price, PricePerSqm, Currency, CityID, Address, Date, Time, MetroID, SettlementsID, Latitude, Longitude)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            
            if not is_listing_exists(listing_id, "EstateListing"):
                try:
                    cursor.execute(estate_query, (listing_id,
                        safe_value(row.get("ListingName")),
                        safe_value(row.get("SalesType")),
                        safe_value(row.get("DailyRent")),
                        safe_value(row.get("CategoryID")),
                        safe_value(row.get("SellerType")),
                        safe_value(row.get("Price")),
                        safe_value(row.get("PricePerSqm")),
                        safe_value(row.get("Currency")),
                        safe_value(row.get("CityID")),
                        safe_value(row.get("Address")),
                        safe_value(row.get("Date")),
                        safe_value(row.get("Time")),
                        safe_value(row.get("MetroID")),
                        safe_value(row.get("SettlementsID")),
                        safe_value(row.get("Latitude")),
                        safe_value(row.get("Longitude"))))
                    stats['estate_listing'] += 1
                except Exception as e:
                    print(f" X EstateListing {listing_id} xətası: {e}")
        conn.commit()


        seller_query = """
        INSERT INTO Dynamic.SellerInfo (ListingID, ListingName, Contact, SellerType, Agency, Residence)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "SellerInfo"):
                try:
                    cursor.execute(seller_query, (
                        listing_id,
                        safe_value(row.get("ListingName")),
                        safe_value(row.get("Contact")),
                        safe_value(row.get("SellerType")),
                        safe_value(row.get("Agency")),
                        safe_value(row.get("Residence"))
                    ))
                    stats['seller_info'] += 1
                except Exception as e:
                    print(f"  ✗ SellerInfo {listing_id} xətası: {e}")

        description_query = """
        INSERT INTO Dynamic.Description (ListingID, Description)
        VALUES (?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Description"):
                try:
                    cursor.execute(description_query, (
                        listing_id,
                        safe_value(row.get("Description"))
                    ))
                    stats['description'] += 1
                except Exception as e:
                    print(f"  ✗ Description {listing_id} xətası: {e}")

        conn.commit()


        if df["CategoryID"].isin([1, 2, 3]).any():
            apartment_query = """
            INSERT INTO Dynamic.Apartment (ListingID, CategoryID, FloorNumber, TotalFloors, FieldAreaSqm, CountRoom, Title, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            df_apartments = df[df["CategoryID"].isin([1, 2, 3])]
            
            for index, row in df_apartments.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Apartment"):
                    try:
                        cursor.execute(apartment_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FloorNumber")),
                            safe_value(row.get("TotalFloors")),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['apartment'] += 1
                    except Exception as e:
                        print(f" X Apartment {listing_id} xətası: {e}")

        if (df["CategoryID"] == 4).any():
            house_query = """
            INSERT INTO Dynamic.House (ListingID, CategoryID, FieldAreaSqm, AreaInSot, CountRoom, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            df_house = df[df["CategoryID"] == 4]
            
            for index, row in df_house.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "House"):
                    try:
                        cursor.execute(house_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("AreaInSot")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['house'] += 1
                    except Exception as e:
                        print(f"X House {listing_id} xətası: {e}")

        if (df["CategoryID"] == 5).any():
            office_query = """
            INSERT INTO Dynamic.Office (ListingID, CategoryID, FieldAreaSqm, [BuildingType (ForOffice)], CountRoom, Title, Repaired)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """
            df_office = df[df["CategoryID"] == 5]
            
            for index, row in df_office.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Office"):
                    try:
                        cursor.execute(office_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("BuildingType (ForOffice)")),
                            safe_value(row.get("CountRoom")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Repaired"))))
                        stats['office'] += 1
                    except Exception as e:
                        print(f" X Office {listing_id} xətası: {e}")

        if (df["CategoryID"] == 6).any():
            garage_query = """
            INSERT INTO Dynamic.Garage (ListingID, CategoryID, FieldAreaSqm, Title)
            VALUES (?, ?, ?, ?)
            """
            df_garage = df[df["CategoryID"] == 6]
            
            for index, row in df_garage.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Garage"):
                    try:
                        cursor.execute(garage_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title"))))
                        stats['garage'] += 1
                    except Exception as e:
                        print(f" X Garage {listing_id} xətası: {e}")


        if (df["CategoryID"] == 7).any():
            land_query = """
            INSERT INTO Dynamic.Land (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage)
            VALUES (?, ?, ?, ?, ?)
            """
            df_land = df[df["CategoryID"] == 7]
            
            for index, row in df_land.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "Land"):
                    try:
                        cursor.execute(land_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")))) 
                        stats['land'] += 1
                    except Exception as e:
                        print(f"X Land {listing_id} xətası: {e}")

        if (df["CategoryID"] == 8).any():
            commercial_query = """
            INSERT INTO Dynamic.CommercialProperty (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage, Repaired)
            VALUES (?, ?, ?, ?, ?, ?)
            """
            df_commercial = df[df["CategoryID"] == 8]
            
            for index, row in df_commercial.iterrows():
                listing_id = safe_value(row["ListingID"])
                if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "CommercialProperty"):
                    try:
                        cursor.execute(commercial_query, (listing_id,
                            safe_value(row["CategoryID"]),
                            safe_value(row.get("FieldAreaSqm")),
                            safe_value(row.get("Title")),
                            safe_value(row.get("Mortgage")),
                            safe_value(row.get("Repaired"))))
                        stats['commercial'] += 1
                    except Exception as e:
                        print(f"X Commercial Property {listing_id} xətası: {e}")

        conn.commit()


        # URL
        url_query = """
        INSERT INTO Dynamic.URL (ListingID, URL)
        VALUES (?, ?)
        """
        
        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing") and not is_listing_exists(listing_id, "URL"):
                try:
                    cursor.execute(url_query, (
                        listing_id,
                        safe_value(row.get("URL"))
                    ))
                    stats['url'] += 1
                except Exception as e:
                    print(f" X URL {listing_id} xətası: {e}")


        labels_query = """
        INSERT INTO Dynamic.Labels (ListingID, Labels)
        VALUES (?, ?)
        """

        def is_listing_label_exists(listing_id, label):
            check_query = "SELECT 1 FROM Dynamic.Labels WHERE ListingID = ? AND Labels = ?"
            cursor.execute(check_query, (listing_id, label))
            return cursor.fetchone() is not None

        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing"):
                labels_raw = row.get("Labels", "")
                
                labels_list = []
                if labels_raw:
                    if isinstance(labels_raw, str):
                        clean_str = str(labels_raw).replace("[", "").replace("]", "").replace("'", "").replace('"', "")
                        labels_list = [x.strip() for x in clean_str.split(",") if x.strip()]
                    elif isinstance(labels_raw, list):
                        labels_list = [str(x).strip() for x in labels_raw if str(x).strip()]
                
                for label in labels_list:
                    if not is_listing_label_exists(listing_id, label):
                        try:
                            cursor.execute(labels_query, (listing_id, label))
                            stats['labels'] += 1
                        except Exception as e:
                            print(f" X Labels {listing_id} ({label}) xətası: {e}")


        def is_image_exists(image_url):
            try:
                check_query = "SELECT COUNT(*) FROM Dynamic.ListingImages WHERE ImageURL = ?"
                cursor.execute(check_query, (image_url,))
                result = cursor.fetchone()
                return result[0] > 0
            except Exception as e:
                print(f"Image yoxlama xətası: {e}")
                return False

        listing_images_query = """
        INSERT INTO Dynamic.ListingImages (ListingID, ImageURL)
        VALUES (?, ?)
        """

        processed_images = set()

        for index, row in df.iterrows():
            listing_id = safe_value(row["ListingID"])
            if is_listing_exists(listing_id, "EstateListing"):
                image_links = row.get("Images")
                
                if image_links and listing_id:
                    if isinstance(image_links, str):
                        try:
                            import json
                            image_list = json.loads(image_links)
                        except:
                            image_list = [link.strip() for link in image_links.split(',') if link.strip()]
                    elif isinstance(image_links, list):
                        image_list = image_links
                    else:
                        image_list = []
                    
                    for image_url in image_list:
                        image_url_clean = safe_value(image_url)
                        
                        if image_url_clean and image_url_clean not in processed_images:
                            if not is_image_exists(image_url_clean):
                                try:
                                    cursor.execute(listing_images_query, (
                                        listing_id,
                                        image_url_clean
                                    ))
                                    stats['listing_images'] += 1
                                    processed_images.add(image_url_clean)
                                except Exception as e:
                                    print(f"X Image {listing_id} xətası: {e}")

        # Final commit
        conn.commit()
        print("Uğurlu")

    except Exception as e:
        print(f"Ümumi xəta baş verdi: {e}")
        try:
            conn.rollback()
            print("Rollback edildi")
        except:
            pass
        return False

    finally:
        try:
            cursor.close()
            conn.close()
            print("Database bağlantısı bağlandı")
        except:
            pass

    # Nəticələr
    print(f"   EstateListing: {stats['estate_listing']}")
    print(f"   SellerInfo:    {stats['seller_info']}")
    print(f"   Description:   {stats['description']}")
    print(f"   Apartment:     {stats['apartment']}")
    print(f"   House:         {stats['house']}")
    print(f"   Office:        {stats['office']}")
    print(f"   Garage:        {stats['garage']}")
    print(f"   Land:          {stats['land']}")
    print(f"   Commercial:    {stats['commercial']}")
    print(f"   URL:           {stats['url']}")
    print(f"   Labels:        {stats['labels']}")
    print(f"   Images:        {stats['listing_images']}")
    print(f"   TOPLAM:        {sum(stats.values())}")

    return True



if __name__ == "__main__":
    Thread(target=scrap, daemon=True).start()
    Thread(target=cleaner, daemon=True).start()
    Thread(target=inserter, daemon=True).start()

    # Prosesi aciq saxlamaq ucun
    raw_data.join()
    cleaned_data.join()

